In [1]:
import numpy as np

def slog2(x):
    if(x <= 0.0):
        return 0.0
    else:
        return np.log2(x)

In [2]:
def load_em_prob_matrix(em_prob_path):
#     lines = open('em_prob.txt').readlines()
    lines = open(em_prob_path).readlines()
    emprob = {}
    
    for i in range(len(lines)):
        lines[i] = lines[i].rstrip('\n')
        lines[i] = lines[i].split()
    
    states = lines[0]
    labels = lines[1]
    lines = lines[2:]
    for i in range(len(states)):
        temp_dict_row = {}
        for j in range(len(labels)):
            temp_dict_row[labels[j]] = float(lines[i][j])
        emprob[states[i]] = temp_dict_row
        
    return emprob

In [3]:
def load_tr_prob_matrix(tr_prob_path):
#     lines = open('tr_prob.txt').readlines()
    lines = open(tr_prob_path).readlines()
    trprob = {}
    
    for i in range(len(lines)):
        lines[i] = lines[i].rstrip('\n')
        lines[i] = lines[i].split()
    
    states = lines[0]
    labels = lines[1]
    lines = lines[2:]
    
    for i in range(len(states)):
        temp_dict_row = {}
        for j in range(len(labels)):
            temp_dict_row[labels[j]] = float(lines[i][j])
        trprob[states[i]] = temp_dict_row
    
    return trprob

In [16]:
def viterbi(obspace, stspace, inprobs, observations, emprobs, trprobs, calc_type='normal'):
    t1 = [[0 for x in range(len(stspace))] for y in range(len(observations)+1)]#initializing 2d matrix for trace
    seq = ''#sequence of states most likely to be followed for observed state
    if(calc_type == 'normal'):
        prob = 1 #I haven't implemented log probability version, therefore prob is 1 and others multiplied on to it
    elif(calc_type == 'log'):
        prob = 0
    
    #initializing initial probabilities
    for i in range(len(stspace)):
        if(calc_type == 'normal'):
            t1[0][i] = inprobs[i]
        elif(calc_type == 'log'):
            t1[0][i] = np.log2(inprobs[i])
    print(t1)
    #for length of observations joint probabilities of switching states for emission are calculated here
    for i in range(1, len(observations)+1):
        for j in range(len(stspace)):#for alpha and beta cross switching
            templ=[]
            for k in range(len(stspace)):
                if(calc_type == 'normal'):
                    print('DEBUG0:',t1[i-1][k])
                    templ.append(t1[i-1][k]*trprobs[stspace[k]][stspace[j]]*emprobs[stspace[k]][observations[i-1]])
                elif(calc_type == 'log'):
                    print('DEBUG0:',t1[i-1][k])
                    print('DEBUG01:',np.log2(trprobs[stspace[k]][stspace[j]]))
                    templ.append(t1[i-1][k]+np.log2(trprobs[stspace[k]][stspace[j]])+np.log2(emprobs[stspace[k]][observations[i-1]]))
            t1[i][j] = max(templ)
  
    #finding index of last state by last observation
    ind = t1[-1].index(max(t1[i]))
    seq+=stspace[ind]
    for i in range(len(observations)-1, -1, -1):
        templ = []
        for j in range(len(stspace)):#scanning switch probabilities from the last index 'ind' and appends it to matrix
            if(calc_type == 'normal'):
                print('DEBUG1:',t1[i][j])
                templ.append(t1[i][j]*trprobs[stspace[j]][stspace[ind]]*emprobs[stspace[j]][observations[i]])
            elif(calc_type == 'log'):
                print('DEBUG1:',t1[i][j])
                print('DEBUG2:',trprobs[stspace[j]][stspace[ind]])
                print('DEBUG3:',emprobs[stspace[j]][observations[i]])
                templ.append(t1[i][j]+np.log2(trprobs[stspace[j]][stspace[ind]])+np.log2(emprobs[stspace[j]][observations[i]]))
    
        maxval = max(templ)
        prev_ind = ind
        ind = templ.index(maxval)
        seq+=stspace[ind]
        
        if(calc_type == 'normal'):
            prob*=(trprobs[stspace[prev_ind]][stspace[ind]]*emprobs[stspace[ind]][observations[i]])#calculates emission and transmission probabilities for that state
        elif(calc_type == 'log'):
            prob+=(np.log2(trprobs[stspace[prev_ind]][stspace[ind]]) + np.log2(emprobs[stspace[ind]][observations[i]]))#calculates emission and transmission probabilities for that state
#         print('TRPROB:',trprobs[stspace[ind]][stspace[prev_ind]],'EMPROB:',emprobs[stspace[prev_ind]][observations[i]],'IND:',ind,"T:",t1[i][j])

#     if(calc_type == 'log'):
#         for i in range(len(observations)+1):
#             for j in range(len(stspace)):
#                 t1[i][j] = slog2(t1[i][j])

    return t1[1:], seq[::-1], prob    # return all together

In [17]:
def assignment3():
    emprobm = load_em_prob_matrix('em_prob.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['A', 'T', 'C', 'G']#observation space
    stspace = ['a', 'b']#state space
    inprobs = [0.5, 0.5]#initial probabilities
    observations = 'GGCT'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='normal')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
assignment3()

[[0.5, 0.5], [0, 0], [0, 0], [0, 0], [0, 0]]
DEBUG0: 0.5
DEBUG0: 0.5
DEBUG0: 0.5
DEBUG0: 0.5
DEBUG0: 0.18000000000000002
DEBUG0: 0.09000000000000001
DEBUG0: 0.18000000000000002
DEBUG0: 0.09000000000000001
DEBUG0: 0.06480000000000001
DEBUG0: 0.016200000000000003
DEBUG0: 0.06480000000000001
DEBUG0: 0.016200000000000003
DEBUG0: 0.005832000000000001
DEBUG0: 0.004374
DEBUG0: 0.005832000000000001
DEBUG0: 0.004374
DEBUG1: 0.005832000000000001
DEBUG1: 0.004374
DEBUG1: 0.06480000000000001
DEBUG1: 0.016200000000000003
DEBUG1: 0.18000000000000002
DEBUG1: 0.09000000000000001
DEBUG1: 0.5
DEBUG1: 0.5
['a', 'b']
[0.18000000000000002, 0.09000000000000001]
[0.06480000000000001, 0.016200000000000003]
[0.005832000000000001, 0.004374]
[0.0005248800000000001, 0.00118098]
Sequence: bbbbb
Probability: 0.002361960000000001


In [13]:
def takehome8():
    emprobm = load_em_prob_matrix('em_prob.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['A', 'T', 'C', 'G']#observation space
    stspace = ['a', 'b']#state space
    inprobs = [0.5, 0.5]#initial probabilities
    observations = 'GTCCATCCACTA'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='normal')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
takehome8()

[[0.5, 0.5], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
DEBUG0: 0.5
DEBUG0: 0.5
DEBUG0: 0.5
DEBUG0: 0.5
DEBUG0: 0.18000000000000002
DEBUG0: 0.09000000000000001
DEBUG0: 0.18000000000000002
DEBUG0: 0.09000000000000001
DEBUG0: 0.016200000000000003
DEBUG0: 0.024300000000000006
DEBUG0: 0.016200000000000003
DEBUG0: 0.024300000000000006
DEBUG0: 0.0014580000000000003
DEBUG0: 0.006561000000000001
DEBUG0: 0.0014580000000000003
DEBUG0: 0.006561000000000001
DEBUG0: 0.00019683000000000005
DEBUG0: 0.0017714700000000002
DEBUG0: 0.00019683000000000005
DEBUG0: 0.0017714700000000002
DEBUG0: 7.085880000000002e-05
DEBUG0: 0.00031886460000000007
DEBUG0: 7.085880000000002e-05
DEBUG0: 0.00031886460000000007
DEBUG0: 9.565938000000003e-06
DEBUG0: 8.609344200000001e-05
DEBUG0: 9.565938000000003e-06
DEBUG0: 8.609344200000001e-05
DEBUG0: 2.58280326e-06
DEBUG0: 2.3245229340000002e-05
DEBUG0: 2.58280326e-06
DEBUG0: 2.3245229340000002e-05
DEBUG0: 6.973568802000001

In [14]:
def takehome9():
    emprobm = load_em_prob_matrix('em_prob.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['A', 'T', 'C', 'G']#observation space
    stspace = ['a', 'b']#state space
    inprobs = [0.5, 0.5]#initial probabilities
    observations = 'CGCCTCGGGC'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='log')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
takehome9()

[[-1.0, -1.0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
DEBUG0: -1.0
DEBUG01: -0.152003093445
DEBUG0: -1.0
DEBUG01: -3.32192809489
DEBUG0: -1.0
DEBUG01: -3.32192809489
DEBUG0: -1.0
DEBUG01: -0.152003093445
DEBUG0: -4.47393118833
DEBUG01: -0.152003093445
DEBUG0: -2.88896868761
DEBUG01: -3.32192809489
DEBUG0: -4.47393118833
DEBUG01: -3.32192809489
DEBUG0: -2.88896868761
DEBUG01: -0.152003093445
DEBUG0: -5.94786237666
DEBUG01: -0.152003093445
DEBUG0: -5.36289987594
DEBUG01: -3.32192809489
DEBUG0: -5.94786237666
DEBUG01: -3.32192809489
DEBUG0: -5.36289987594
DEBUG01: -0.152003093445
DEBUG0: -9.421793565
DEBUG01: -0.152003093445
DEBUG0: -7.25186856355
DEBUG01: -3.32192809489
DEBUG0: -9.421793565
DEBUG01: -3.32192809489
DEBUG0: -7.25186856355
DEBUG01: -0.152003093445
DEBUG0: -12.3107622526
DEBUG01: -0.152003093445
DEBUG0: -9.14083725117
DEBUG01: -3.32192809489
DEBUG0: -12.3107622526
DEBUG01: -3.32192809489
DEBUG0: -9.14083725117
DEBUG01: -0.152003093445

In [18]:
def takehome10():
    emprobm = load_em_prob_matrix('em_prob_final.txt')#loading emission probability matrix into a 2d dictionary
    trprobm = load_tr_prob_matrix('tr_prob_final.txt')#loading transmission probability matrix into a 2d dictionary
    obspace = ['1', '2', '3']#observation space
    stspace = ['a', 'b', 'c']#state space
    inprobs = [0.5, 0.5, 1.0]#initial probabilities
    observations = '112122'#observed sequence

    m,s,p = viterbi(obspace, stspace, inprobs, observations, emprobm, trprobm, calc_type='log')

    print(stspace)#printing states
    for sm in m:
        print(sm)#printing joing probabilities used to determine path

    print('Sequence:',s)
    print('Probability:',p)
    print(emprobm)
    print(trprobm)
takehome10()

[[-1.0, -1.0, 0.0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0]]
DEBUG0: -1.0
DEBUG01: -1.0
DEBUG0: -1.0
DEBUG01: -2.0
DEBUG0: 0.0
DEBUG01: -inf
DEBUG0: -1.0
DEBUG01: -2.0
DEBUG0: -1.0
DEBUG01: -1.0
DEBUG0: 0.0
DEBUG01: -inf
DEBUG0: -1.0
DEBUG01: -2.0
DEBUG0: -1.0
DEBUG01: -2.0
DEBUG0: 0.0
DEBUG01: -inf
DEBUG0: -3.0
DEBUG01: -1.0
DEBUG0: -4.0
DEBUG01: -2.0
DEBUG0: -4.0
DEBUG01: -inf
DEBUG0: -3.0
DEBUG01: -2.0
DEBUG0: -4.0
DEBUG01: -1.0
DEBUG0: -4.0
DEBUG01: -inf
DEBUG0: -3.0
DEBUG01: -2.0
DEBUG0: -4.0
DEBUG01: -2.0
DEBUG0: -4.0
DEBUG01: -inf
DEBUG0: -5.0
DEBUG01: -1.0
DEBUG0: -6.0
DEBUG01: -2.0
DEBUG0: -6.0
DEBUG01: -inf
DEBUG0: -5.0
DEBUG01: -2.0
DEBUG0: -6.0
DEBUG01: -1.0
DEBUG0: -6.0
DEBUG01: -inf
DEBUG0: -5.0
DEBUG01: -2.0
DEBUG0: -6.0
DEBUG01: -2.0
DEBUG0: -6.0
DEBUG01: -inf
DEBUG0: -8.0
DEBUG01: -1.0
DEBUG0: -8.0
DEBUG01: -2.0
DEBUG0: -9.0
DEBUG01: -inf
DEBUG0: -8.0
DEBUG01: -2.0
DEBUG0: -8.0
DEBUG01: -1.0
DEBUG0: -9.0
DEBUG01: -inf
DEBUG0: -8.0
DEBUG01: -2.0

/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:26: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:27: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.6/site-packages/ipykernel/__main__.py:43: RuntimeWarning: divide by zero encountered in log2
